# Функции: избранные темы

## Вступление: контекстные менеджеры

В Python по умолчанию задана некоторая точность вычисления чисел с плавающей точкой, которая, что логично, может отличаться от требуемой для решения конкретной задачи точности. Для пример, будет рассматривать точность вычислений в библиотеке **decimal**.

Предположим, нас устраивает точность вычислений по умолчанию в этой библиотеке, но для некоторых расчетов мы бы хотели использовать повышенную или пониженную точность и только для них. Как нам быть? Тут-то нам на помощь и придут контекстные менеджеры.

В этом примере мы поговорим о контекстных менеджерах и напишем свой менеджер для установки точности вычислений, которые будет работать примерно так:

```python
with set_precision(5):
    ...
```

In [32]:
from decimal import Decimal, getcontext
from contextlib import contextmanager


# наш код
@contextmanager
def set_precision(n: int):
    ## start
    start_prec = getcontext().prec
    getcontext().prec = n
    try:
        yield             # = next
    finally:
        getcontext().prec = start_prec
        ## end

with set_precision(5):
    print(Decimal(5) / Decimal(3))

# __start__
# __next__
# __stop__

1.6667


## Разминка: геометрическая прогрессия

Напишите бесконечный генератор геометрической прогрессии. В качестве параметров генератор должен принимать:  
- первый член прогрессии
- шаг прогрессии

In [4]:
from typing import Generator
# ваш код

def geom_progr(start = 1, step = 2) -> Generator:
    curr = start
    while True:
        yield curr
        curr *= step


my_geom_progr = geom_progr()

for i in range(10):
    print(next(my_geom_progr))
    # next(my_geom_progr)

1
2
4
8
16
32
64
128
256
512


## Задание 1: float_range

Необходимо создать аналог range(), который генерирует арифметическую прогрессию. Ниже приведены примеры использования:

*Пример 1*:

```python
for i in float_range(stop=5):
    print(i)
```

*Вывод*:
```console
0
1
2
3
4
```
___
*Пример 2*:

```python
for i in float_range(stop=1, step=0.5):
    print(i)
```

*Вывод*:
```console
0
0.5
```
___
*Пример 3*:

```python
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
```

*Вывод*:
```console
1
0.5
0
-0.5
```

In [73]:
def float_range(start = 0, stop = 10, step = 1):
    curr = start
    while True:
        if (curr <=stop and step > 0) or (curr >= stop and step <0):
            yield curr
            curr += step  
        else:
            return    
  
for i in float_range(stop=5):
    print(i)

print()

for i in float_range(stop=1, step=0.5):
    print(i)

print()

for i in float_range(start = 1, stop=-1, step=0):
    print(i)

0
1
2
3
4
5

0
0.5
1.0



## Задание 2: свой map

### Часть 1: копируем map

Реализуйте аналог функции map, полностью копирующий ее поведение. Саму map использовать нельзя.

In [75]:
# ваш код

# def my_map(func, *s:list) -> list:
#     ans = []
#     for i in range(min([len(_) for _ in s])):
#         needed_elems = []
#         for spis in s:
#             curr = spis[i]
#             needed_elems.append(curr)
#         ans.append(func(*needed_elems))
#     return ans

def my_map(func, *s:list) -> list:
    ans = []
    for i in zip(*s):
        
        ans.append(func(*i))
    return ans
s = [1,2,3,4,5]

kv_s = my_map(lambda x: x**2, s)
print(kv_s)

s1 = [2,3,4,5,6]

razn = my_map(lambda x, y: x-y, s1, s)
print(razn)

for i in kv_s:
    print(i)



[1, 4, 9, 16, 25]
[1, 1, 1, 1, 1]
1
4
9
16
25


### Часть 2: дополняем map

Добавьте возможность управлять поведением вашего map'a: сделайте так, чтобы map имела возможность не только обрезать последовательности, но и дополнять короткие последовательности до динных. 

Совет: функция **zip_longest** из библиотеки **itertools** может оказаться полезной.

In [62]:
from enum import Enum
from itertools import zip_longest


class MapTypes(Enum):
    SHORTEST = 'short'
    LONGEST = 'long'


In [117]:
def my_map2(func, *s:list, map_type=MapTypes.SHORTEST) -> Generator:
    ans = []

    if map_type == MapTypes.SHORTEST:
        for i in zip(*s):
            yield func(*i)
    elif map_type == MapTypes.LONGEST:
        for i in zip_longest(*s):
            yield func(*i)

s = [1,2,3,4]

# kv_s = my_map2(lambda x: x**2, s)
# print(kv_s)

s1 = [2,3,4,5,6]

# razn = my_map2(lambda x, y: x-y, s1, s)
# print(razn)

# razn = my_map2(lambda x, y: x-y, s1, s, map_type=MapTypes.LONGEST)
# print(razn)
m = my_map2(lambda x:x**2, s)
for i in range(4):
    print(next(m))


1
4
9
16


## Задание 3: Спиннер

### Часть 1: генератор

Напишите функцию, которая на вход получает коллекцию и возвращает генератор, последовательно возвращающий элементы коллекции, а после возврата последнего элемента коллекции очередной вызов генератора приведет к зацикливанию.


*Пример*:

```python
generator = generate_circle('abc')

print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
```
*Вывод*:
```console
a
b
c
a
b
```

In [84]:
def generate_circle(s:str) -> Generator:
    curr = 0
    while True:
        if curr<len(s):
            yield s[curr]
            curr += 1  
        else:
            curr = 0
            yield s[curr]

g = generate_circle("abc")
for i in range(10):
    print(next(g))

a
b
c
a
a
b
c
a
a
b


### Часть 2: Колесо Сансары

Используя генератор из предыдущего раздела, реализуйте функцию, которая отображает на экране спиннер для индикации загрузки.

Отрисовка спиннера печатает на экран надпись: Thinking: \<symbol\>, где вместо \<symbol\> последовательно появляются знаки: \, |, /, -, что создаёт эффект вращения.

Вход функции: 
- time_limit - время (в секундах), в течение которого должна производиться отрисовка спиннера;
- pause - время (в секундах) задержки между сменой символов спиннера;

Интересная статья на тему индикаторов: https://dtf.ru/flood/174240-progress-bar-ili-spinner-chto-i-kogda-ispolzovat?ysclid=lorrg51syv550654720

In [126]:
import time
import os

def generate_circle(time_limit, pause) -> Generator:
    s = '\|/-'
    curr = 0
    timee = pause
    while timee<=time_limit:
        if curr<len(s):
            time.sleep(pause)
            yield s[curr]
            curr += 1  
            timee+=pause
        else:
            curr = 0

            time.sleep(pause)

            yield s[curr]
            curr+=1
            timee+=pause
    return

for i in generate_circle(10,1):
    print(f"\rthinking: {i}", end = "")


thinking: |

\